# Data Clean-up and Wrangling of Amazon Product Reviews Dataset

## Data Wrangling 

The Amazon Customer Reviews and Product Metadata can be found [here](https://nijianmo.github.io/amazon/index.html). 
The Dataset is an updated version of the Amazon review dataset realease in 2014.  This dataset includes reviews (ratings, text, helpfulness votes), product metadata (descriptions, category information, price, brand, and image features). For each product category there are 2 different datasets that can be dowbload: 

- reviews: raw review data
- metadata: includes product descriptions, price, sales-rank, and co-purchasing links.

**Reviews**
- reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- asin - ID of the product, e.g. 0000013714
- reviewerName - name of the reviewer **(Deleted)**
- vote - helpful votes of the review **(Deleted)**
- style - a dictionary of the product metadata, e.g., "Format" is "Hardcover" **(Deleted)**
- reviewText - text of the review
- overall - rating of the product
- summary - summary of the review
- unixReviewTime - time of the review (unix time) **converted to readable date and renamed to 'reviewDate'**
- reviewTime - time of the review (raw) **(Deleted)**
- image - images that users post after they have received the product **(Deleted)**
- month - the month the product was reviewed - extracted from reviewDate **(Added)**
- year - the year the product was reviewed - extracted from reviewDate **(Added)**

**metadata**
- asin - ID of the product, e.g. 0000031852
- title - name of the product
- feature - bullet-point format features of the product
- description - description of the product
- price - price in US dollars (at time of crawl)
- image - url of the product image
- related - related products (also bought, also viewed, bought together, buy after viewing)
- salesRank - sales rank information
- brand - brand name
- categories - list of categories the product belongs to **(Deleted)**
- tech1 - the first technical detail table of the product
- tech2 - the second technical detail table of the product
- similar - similar product table
- fit - all rows contained empty spaces **(Deleted)**

We found that:
- The Appliances dataset contains 602,777 rows and 12 columns
- image, vote and style columns had 98%, 89% and 77% nulls values respectively - therefore those columns were deleted. 
- reviewText has 0.05% missing values - the rows with missing values will be deleted as they will not affect our dataset. 
- summary has 0.02% missing values - the rows with missing values will be deleted as they will not affect our dataset. 

In [3]:
# import required libraries 
import pandas as pd
import numpy as np
import json
import gzip
import os
import re

In [4]:
# load the product reviews dataset
review_df = pd.read_json("data/Appliances.json.gz", compression='gzip',lines=True)   
print('Total number of reviews:' + str(len(review_df)))


Total number of reviews:602777


In [5]:
print('Number of rows and columns in dataset: ' + str(review_df.shape))

Number of rows and columns in dataset: (602777, 12)


In [6]:
#check what the data looks like 
review_df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,2,False,"11 27, 2013",A3NHUQ33CFH3VM,1118461304,{'Format:': ' Hardcover'},Greeny,Not one thing in this book seemed an obvious o...,Clear on what leads to innovation,1385510400,NaN
1,5,NaN,False,"11 1, 2013",A3SK6VNBQDNBJE,1118461304,{'Format:': ' Kindle Edition'},Leif C. Ulstrup,I have enjoyed Dr. Alan Gregerman's weekly blo...,Becoming more innovative by opening yourself t...,1383264000,NaN
2,5,NaN,False,"10 10, 2013",A3SOFHUR27FO3K,1118461304,{'Format:': ' Hardcover'},Harry Gilbert Miller III,Alan Gregerman believes that innovation comes ...,The World from Different Perspectives,1381363200,NaN
3,5,NaN,False,"10 9, 2013",A1HOG1PYCAE157,1118461304,{'Format:': ' Hardcover'},Rebecca Ripley,"Alan Gregerman is a smart, funny, entertaining...",Strangers are Your New Best Friends,1381276800,NaN
4,5,10,False,"09 7, 2013",A26JGAM6GZMM4V,1118461304,{'Format:': ' Hardcover'},Robert Morris,"As I began to read this book, I was again remi...","How and why it is imperative to engage, learn ...",1378512000,NaN


From the above data we can see the following:

- 'vote' maybe contains alot of NaN
- 'reviewTime' and 'unixReviewTime' probably offer the same information
- 'style' doesn't seem to offer valuable information
- 'reviewerName' is not needed since we have reviwerID
- 'image' is also not needed since we will not be analyzing any images from reviewers

Therefore, these columns will be dropped upon further investigation.

In [7]:
#check the review ranges - should be 1 to 5
#check the frequency of each rating
print('Unique Reviews:')
print(review_df['overall'].unique())
print('\n')
print(review_df['overall'].value_counts())

Unique Reviews:
[5 4 3 1 2]


5    416288
4     75476
1     59627
3     30652
2     20734
Name: overall, dtype: int64


In [8]:
#calculate the number of NULL values and their % across all columns 
missing = pd.concat([review_df.isnull().sum(), 100 * review_df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending=False)

,count,%
image,593519,98.464109
vote,537515,89.173110
style,464804,77.110441
reviewText,324,0.053751
summary,128,0.021235
reviewerName,15,0.002488
overall,0,0.000000
verified,0,0.000000
reviewTime,0,0.000000
reviewerID,0,0.000000


In [9]:
#delete style, reviewrName, image
#drop vote column since it's not needed and has 537515 NaN values
#create a list of the columns we want to keep
col = ['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewText', 'summary', 'unixReviewTime']
review_df = review_df[col]

#### Cleanup the date columns

In [10]:
#conver unixreviewtime to datetime dtype
from datetime import datetime

condition = lambda row: datetime.fromtimestamp(row).strftime('%m-%d-%Y')
review_df['unixReviewTime'] = review_df['unixReviewTime'].apply(condition)

#rename unixReviewTime to reviewDate
review_df.rename(columns={'unixReviewTime': 'reviewDate'}, inplace=True)

#delete reviewTime column 
del review_df['reviewTime']

In [11]:
# add column for month and yead extracted from reviewDate column
review_df['year'] = pd.DatetimeIndex(review_df['reviewDate']).year
review_df['month'] = pd.DatetimeIndex(review_df['reviewDate']).month

In [12]:
#check the columns in the dataset
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602777 entries, 0 to 602776
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   overall     602777 non-null  int64 
 1   verified    602777 non-null  bool  
 2   reviewerID  602777 non-null  object
 3   asin        602777 non-null  object
 4   reviewText  602453 non-null  object
 5   summary     602649 non-null  object
 6   reviewDate  602777 non-null  object
 7   year        602777 non-null  int64 
 8   month       602777 non-null  int64 
dtypes: bool(1), int64(3), object(5)
memory usage: 37.4+ MB


#### Delete duplicates and Nulls

reviewText and summary columns may be used for NLP if time allows - therefore we will drop the rows with NA values - only 0.05 for review test and 0.02% for summary - both combined if the missing columns don't overlap will decrease our dataset by 0.07%

In [13]:
#delete rows with Null values from reviewText and summary
review_df.dropna(subset=['reviewText','summary'], inplace=True)

In [14]:
#check how many duplicate rows
print('Number of duplicate rows:' + str(review_df.duplicated().sum()))

Number of duplicate rows:11423


In [15]:
#drop duplicates
review_df = review_df.drop_duplicates()

In [16]:
#reset index after dropping duplicates
review_df = review_df.reset_index(drop = True)

In [17]:
review_df.shape

(590917, 9)

#### Extract values from other columns 

In [18]:
#how many times was each product(asin) reviewed?
review_df.asin.value_counts()

B000AST3AK    6509
B004UB1O9Q    5698
B00KJ07SEM    3199
B0045LLC7K    2933
B001B35APA    2481
              ... 
B00BLDHLN4       1
B01A9K0X9I       1
B001HOIZO6       1
B00LOX1NZS       1
B007QAKXV8       1
Name: asin, Length: 30241, dtype: int64

In [19]:
#calculate mean average for each product
review_df.groupby('asin')['overall'].mean()

asin
1118461304    4.600000
1906487049    4.666667
6040985461    5.000000
7301113188    5.000000
7861850250    3.000000
                ...   
B01HJH651Y    3.666667
B01HJH6JT2    5.000000
B01HJH92JQ    2.000000
B01HJHHEA0    3.500000
B01HJHHQM6    5.000000
Name: overall, Length: 30241, dtype: float64

Having the total number of reviews for each product and the overall rating for all reviews will be useful so we will add those columns 

In [20]:
#caculate reviewCount for each product and merge the data to review_df
reviewCount = review_df.asin.value_counts().rename('reviewCount')
review_df = review_df.merge(reviewCount.to_frame(), left_on = 'asin', right_index=True)

In [21]:
#caculate avgReview for each product and merge the data to review_df
avgReview = review_df.groupby('asin')['overall'].mean().rename('avgReview')
review_df = review_df.merge(avgReview.to_frame(), left_on = 'asin', right_index=True)

#### Some more tidying 

Here we'll delete all rows with less than 50 reviews - but first we'll check how much of our data will be lost by doing so. 

In [23]:
review_df[review_df['reviewCount'] >= 20].count()

overall        495819
verified       495819
reviewerID     495819
asin           495819
reviewText     495819
summary        495819
reviewDate     495819
year           495819
month          495819
reviewCount    495819
avgReview      495819
dtype: int64

Deleting all reviews for products with less than 50 will still allow us to retain 84% of our dataset and we end up with 495,819 rows. 

In [24]:
review_df = review_df[review_df['reviewCount'] >= 20]

In [25]:
review_df.shape

(495819, 11)

In [27]:
review_df = review_df.reset_index(drop = True)

In [28]:
review_df.sample(10)

,overall,verified,reviewerID,asin,reviewText,summary,reviewDate,year,month,reviewCount,avgReview
70725,4,True,A2YIBBPK6JFVQC,B001AAEG6S,"I rated this 4 stars out of 5 because, as othe...",Gets the job done,12-14-2013,2013,12,330,4.087879
91575,1,True,A2W0SON0J5MTHX,B001TH7H04,The ones I received were not the same as the o...,The ones I received were not the same as the ...,11-07-2016,2016,11,1665,3.971772
368550,5,True,AA0QHPK9T7LA2,B00QFVF3IC,"No issues, works great with my fridge.",works great with my fridge,09-09-2017,2017,9,303,4.716172
363715,5,True,A3ORXEXUEMRWCG,B00OP8991Y,No more leaks,Worked as expected,03-17-2015,2015,3,450,4.540000
234683,4,True,A13O00R43MQJ1C,B00CDWTQKI,It's like a salad spinner for clothes.,Does what it's supposed to.,01-31-2017,2017,1,625,4.513600
4433,5,True,A3IA6MZ6LPX12Q,B0000CFPK8,Cock a doodle do! Love the burner covers...Loo...,Cock a doodle do!,05-03-2015,2015,5,1262,4.358954
288987,5,True,A2BA4PXOJTLTV1,B00ES4XO98,Fits like a dream in my fridge! Took longer to...,"It's a switch, and it works.",08-08-2015,2015,8,99,4.646465
366100,5,True,A11O9H4ACDLO9H,B00PGFGVB6,This product works great! It is designed to cr...,Great product!,04-01-2015,2015,4,122,2.959016
267372,5,True,AX263PUA0NU9A,B00DZU8M9I,Exactly what I needed.,Five Stars,11-01-2016,2016,11,349,4.638968
495074,5,True,A26E9ODCYOLF0D,B01HBPGP28,"Great replacement for my GE refrigerator , A ...",Great replacement for my GE refrigerator,04-16-2017,2017,4,482,4.502075
